# Open Crime Data Plot

This notebook produces a stacked-area plot of crime by month. It uses the more comprehensive open data [published](https://geodash.vpd.ca/opendata/) by the VPD.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
neighbourhood = "Vancouver"

In [ ]:
# df = pd.read_csv("crime-data.csv")

# df.to_parquet("crime-data.parquet")

df = pd.read_parquet("crime-data.parquet")

In [ ]:
if neighbourhood in df["NEIGHBOURHOOD"].unique():
    df = df[df["NEIGHBOURHOOD"] == neighbourhood]
final_df = df.groupby(["YEAR", "MONTH", "TYPE"])[["DAY"]].count().rename(
    columns={"DAY": "Reported Crimes"},
    index={
        1: "January",
        2: "February",
        3: "March",
        4: "April",
        5: "May",
        6: "June",
        7: "July",
        8: "August",
        9: "September",
        10: "October",
        11: "November",
        12: "December",
    }
).reset_index().rename(columns={"TYPE": "Category"})

final_df["Date"] =  pd.to_datetime(final_df["MONTH"] + "-" + final_df["YEAR"].astype(str)) + pd.tseries.offsets.MonthEnd(1)
final_df = final_df.drop(columns=["MONTH", "YEAR"])

In [ ]:
# px.colors.sequential.Teal
# px.colors.sequential.Purp
# px.colors.sequential.Blues

In [ ]:
group = {
    "Vehicle Collision or Pedestrian Struck (with Injury)": "Violent Crime",
    "Vehicle Collision or Pedestrian Struck (with Fatality)": "Violent Crime",
    "Offence Against a Person": "Violent Crime",
    "Homicide": "Violent Crime",
    "Break and Enter Commercial": "Property Crime", 
    "Break and Enter Residential/Other": "Property Crime",
    "Theft from Vehicle": "Property Crime",
    "Theft of Vehicle": "Property Crime",
    "Theft of Bicycle": "Property Crime",
    "Other Theft": "Property Crime",
    "Mischief": "Property Crime",
    }

In [ ]:
fig = px.area(
    final_df, 
    x="Date", 
    y="Reported Crimes", 
    color="Category", 
    title=f"<b>Crime in {neighbourhood}</b> <br><a href='https://geodash.vpd.ca/opendata/'>Source: Vancouver Police Department</a>",
    template="ggplot2",
    color_discrete_map={
        "Vehicle Collision or Pedestrian Struck (with Injury)": 'rgb(228, 199, 241)',
        "Vehicle Collision or Pedestrian Struck (with Fatality)": 'rgb(185, 152, 221)',
        "Offence Against a Person": 'rgb(159, 130, 206)',
        "Homicide": 'rgb(99, 88, 159)',
        "Mischief": 'rgb(209, 238, 234)',
        "Other Theft": 'rgb(168, 219, 217)',
        "Theft from Vehicle": 'rgb(133, 196, 201)', 
        "Break and Enter Commercial": 'rgb(104, 171, 184)',
        "Break and Enter Residential/Other": 'rgb(79, 144, 166)',
        "Theft of Vehicle": 'rgb(59, 115, 143)',
        "Theft of Bicycle": 'rgb(42, 86, 116)',
        
    },
    category_orders={
        "Category": [
            "Vehicle Collision or Pedestrian Struck (with Injury)",
            "Vehicle Collision or Pedestrian Struck (with Fatality)",
            "Offence Against a Person",
            "Homicide",
            "Mischief",
            "Other Theft",
            "Theft from Vehicle",
            "Break and Enter Commercial", 
            "Break and Enter Residential/Other",
            "Theft of Vehicle",
            "Theft of Bicycle",
        ]
    },
)
# use the legend grouping
# for trace in fig.data:
#     trace["legendgroup"] = group[trace.name]
#     trace["legendgrouptitle_text"]=group[trace.name]

fig.add_trace(
    go.Scatter(
        x=['2020-03-17','2020-03-17'],
        y=[0,3400],
        mode='lines',
        line=dict(dash='dash', color="grey"), 
        name='COVID-19 Emergency Declared',
        visible=False,
    )
)

fig.update_xaxes(
    rangeslider=dict(
            visible=True
        ),
    rangeselector=dict(
            buttons=list([
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=3,
                     label="3y",
                     step="year",
                     stepmode="backward"),
                dict(count=5,
                     label="5y",
                     step="year",
                     stepmode="backward"),
                dict(count=10,
                     label="10y",
                     step="year",
                     stepmode="backward"),
                dict(step='all')
            ])
        ),
    range=["2018-02-01", "2022-04-30"],
    type="date")
fig.write_html('index.html')
# fig.write_image("crime_plot.png", height=1000, width=1500)
fig.update_layout(height=800)
fig.show()